# Expense recorder for my Journey Journal
This won't quite work the way I intended

In [83]:
import requests
from flask import jsonify
import json
from dash import Dash, html, dcc, callback, Output, Input, State, dash_table
import pandas as pd
from datetime import datetime, date
import plotly.express as px
from urllib.parse import quote

# Set base URL for API calls
url = 'http://localhost:5000/'

# We'll use this to append to the base URL for specific API calls
url_part = ''

# We'll reserve the payload for POST/PUT methods generally
payload = {}

# We don't have any specific headers, but this can be used for authentication if required
headers = {}

In [98]:
def get_trips(keyword=None, start_date=date(1900, 1, 1), end_date=(date(2099, 12, 31))):
        
    # Add parameters to the URL if they are provided
    url_parameters = {}
    if keyword is not None:
        url_parameters['query'] = keyword
    if start_date is not None:
        start_date_string = start_date.strftime('%Y-%m-%d')
        url_parameters['start_date'] = start_date_string
    if end_date is not None:
        end_date_string = end_date.strftime('%Y-%m-%d')
        url_parameters['end_date'] = end_date_string
    
    # If the parameters are all empty, use the base /trips endpoint to get all trips
    # Otherwise, use the /trips/search endpoint to get trips matching the parameters
    if not any(url_parameters.values()):
        url_part = '/trips'
    else:
        url_part = 'trips/search?' + '&'.join([f'{k}={v}' for k, v in url_parameters.items()])

    print(url_part)
    print(url + url_part)

    response = requests.request("GET", url + url_part, headers=headers, data=payload)
    trips = response.json()
    return trips

def get_trips_by_keyword(keyword=None):
    
    # If keyword is provided, use the trips/search endpoint to search for trips matching the keyword
    # Otherwise the base /trips endpoint will be used to return all trips 
    if keyword is not None:
        keyword_string = quote(keyword)
        url_part = 'trips/search?query=' + keyword_string
    else:
        url_part = '/trips'

    response = requests.request("GET", url + url_part, headers=headers, data=payload)
    trips = response.json()

    print(trips)

    return trips

def get_trips_by_traveler(traveler=None):

    url_part = '/trips'
    
    # If traveler is provided, append the parameter to the search
    # Otherwise the base /trips endpoint will be used to return all trips 
    if traveler is not None:
        traveler_string = quote(traveler)
        url_part = url_part + '?traveler=' + traveler_string

    response = requests.request("GET", url + url_part, headers=headers, data=payload)
    trips = response.json()
    return trips

Navigate to a trip

In [36]:
# Ask for a trip ID (this will be replaced with a dropdown in the app)
trip_id = input("Enter trip id: ")

# Get the trip information
url_part = 'trips/' + str(trip_id)
response = requests.request("GET", url + url_part, headers=headers, data=payload)
trip = response.json()

# Get the trip information and format a trip title
destination = trip['destination']
start_date = datetime.strptime(trip['start_date'], '%a, %d %b %Y %H:%M:%S %Z') # Convert to datetime object
end_date = datetime.strptime(trip['end_date'], '%a, %d %b %Y %H:%M:%S %Z') # Convert to datetime object

trip_title = destination + ': ' + start_date.strftime('%b %#d, %Y') + ' - ' + end_date.strftime('%b %#d, %Y')

# Get the trip's expenses
url_part = 'trips/' + str(trip_id) + '/expenses'
response = requests.request("GET", url + url_part, headers=headers, data=payload)
expenses = response.json()

In [ ]:
# Get a trip's full details by id
def get_trip_details(trip_id):
    url_part = '/trips' + str(trip_id) + '/details'
    response = requests.request("GET", url + url_part, headers=headers, data=payload)
    trip = response.json()

    # Calculate total cost
    

    return trip

In [132]:


app = Dash(__name__)

app.layout = html.Div([
    html.Div(children=[
        html.H1('Browse Trips'),
        dcc.Input(
            id='my-keyword-input',
            type='text',
            placeholder='Keyword...'
            ),
        html.Button(
            id='keyword-submit-button-state',
            n_clicks=0,
            children='Submit'
            ),
        html.Div(id='list-output-state')
    ],
    style={'padding': 10, 'flex': 1}
    ),

    html.Div(children=[
        html.H1('Trip Details'),
        dcc.Input(
            id='trip-id-input',
            type='number',
            placeholder='Trip ID...'
            ),
        html.Button(
            id='trip-submit-button-state',
            n_clicks=0,
            children='Submit'
        ),
        html.Div(id='trip-output-state')
    ],
    style={'padding': 10, 'flex': 1}
    )
],
style={'display': 'flex', 'flexDirection': 'row'}
)

# Callbacks to search by keyword and create a list of trips
@callback(
    Output('list-output-state', 'children'),
    Input('keyword-submit-button-state', 'n_clicks'),
    State('my-keyword-input', 'value')
)
def update_figure(n_clicks,value):
    
    # Get the trips and convert to a dataframe
    trips = get_trips_by_keyword(value)

   # Convert dates to date objects
    for trip in trips:
        # Convert dates to date objects
        trip['start_date'] = datetime.strptime(trip['start_date'], '%a, %d %b %Y %H:%M:%S %Z')
        trip['end_date'] = datetime.strptime(trip['end_date'], '%a, %d %b %Y %H:%M:%S %Z')

    # Create an HTML div for each trip
    output_details = []
    for trip in trips:
        # Create a trip title

        if trip['destination'] is None:
            destination = 'No Destination'
        else:
            destination = trip['destination']

        start_date = trip['start_date']
        end_date = trip['end_date']

        if start_date.year == end_date.year and start_date.month == end_date.month:
            trip_title = f'{destination}: {start_date.strftime("%b %Y")}'
        elif start_date.year == end_date.year:
            trip_title = f'{destination}: {start_date.strftime("%b")}-{end_date.strftime("%b %Y")}'
        else:
            trip_title = f'{destination}: {start_date.strftime("%b %Y")} - {end_date.strftime("%b %Y")}'
        
        # Create an HTML div for each trip
        output_details.append(html.Div(
            children=[
                html.H4(trip_title),
                html.P(f"{trip['start_date'].strftime('%b %#d, %Y')} - {trip['end_date'].strftime('%b %#d, %Y')} ({trip['duration_days']} days)",
                    style={'fontStyle': 'italic'}),
                html.P(f"Traveler: {trip['traveler_name']} ({trip['traveler_gender']}, {trip['traveler_age']})"),
                html.P("Nationality: " + trip['traveler_nationality']),
                html.P(children=["Comment: ", trip['trip_comment']]),
                html.P(children=["Id: ", trip['id']]),
                # html.Button(
                #     id={'type': 'view-details-button', 'index': trip['id']},
                #     children='View & Edit',
                #     n_clicks=0
                # ),
                html.Hr()
            ]
            )
        )

    # Define the header text
    if value is None:
        header_text = 'All Trips'
    else:
        header_text = f"Trips Matching '{value}'"
    
    # Format and return the output

    output = html.Div([
        html.H3(header_text),
        html.Div(output_details)
    ])

    return output

# Callback to get trip details based on the id of the trip that was clicked or entered
@callback(
    Output('trip-output-state', 'children'),
    Input({'type': 'view-details-button', 'index': dash.dependencies.ALL}, 'n_clicks'),
    State({'type': 'view-details-button', 'index': dash.dependencies.ALL}, 'id'),
    State('trip-id-input', 'value')
)
def get_trip_details(n_clicks, button_id, trip_id):


if __name__ == '__main__':
    app.run(debug=True)

[{'destination': 'London, UK', 'duration_days': 7, 'end_date': 'Mon, 08 May 2023 00:00:00 GMT', 'id': 1, 'start_date': 'Mon, 01 May 2023 00:00:00 GMT', 'traveler_age': 35, 'traveler_gender': 'Male', 'traveler_name': 'John Smith', 'traveler_nationality': 'American', 'trip_comment': None}, {'destination': 'Phuket, Thailand', 'duration_days': 5, 'end_date': 'Tue, 20 Jun 2023 00:00:00 GMT', 'id': 2, 'start_date': 'Thu, 15 Jun 2023 00:00:00 GMT', 'traveler_age': 28, 'traveler_gender': 'Female', 'traveler_name': 'Jane Doe', 'traveler_nationality': 'Canadian', 'trip_comment': None}, {'destination': 'Bali, Indonesia', 'duration_days': 7, 'end_date': 'Sat, 08 Jul 2023 00:00:00 GMT', 'id': 3, 'start_date': 'Sat, 01 Jul 2023 00:00:00 GMT', 'traveler_age': 45, 'traveler_gender': 'Male', 'traveler_name': 'David Lee', 'traveler_nationality': 'Korean', 'trip_comment': None}, {'destination': 'New York, USA', 'duration_days': 14, 'end_date': 'Tue, 29 Aug 2023 00:00:00 GMT', 'id': 4, 'start_date': 'Tue,